<a href="https://www.kaggle.com/code/inderjeet21/nlp-pipeline-imdb?scriptVersionId=253724104" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-movie-review-dataset/movie_data.csv
/kaggle/input/harry-potter-books/03 Harry Potter and the Prisoner of Azkaban.txt
/kaggle/input/harry-potter-books/06 Harry Potter and the Half-Blood Prince.txt
/kaggle/input/harry-potter-books/05 Harry Potter and the Order of the Phoenix.txt
/kaggle/input/harry-potter-books/02 Harry Potter and the Chamber of Secrets.txt
/kaggle/input/harry-potter-books/07 Harry Potter and the Deathly Hallows.txt
/kaggle/input/harry-potter-books/01 Harry Potter and the Sorcerers Stone.txt
/kaggle/input/harry-potter-books/04 Harry Potter and the Goblet of Fire.txt


# NLP Pipeline


let’s cover the NLP(Natural Language Processing) pipeline, which is a core concept in both classical and modern NLP. It's the step-by-step process used to turn raw text into structured data or actionable insights.

# 1. Text Preprocessing


In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv("/kaggle/input/imdb-movie-review-dataset/movie_data.csv")
df.head()

,review,sentiment
0,This movie is just crap. Even though the direc...,0
1,Another detailed work on the subject by Dr Dwi...,1
2,THE CAT O'NINE TAILS (Il Gatto a Nove Code) <b...,0
3,"Like with any movie genre, there are good gang...",0
4,I watched it with my mom and we were like...<b...,0


In [3]:
df.shape

(49969, 2)

In [4]:
df['review'].str.lower()

0        this movie is just crap. even though the direc...
1        another detailed work on the subject by dr dwi...
2        the cat o'nine tails (il gatto a nove code) <b...
3        like with any movie genre, there are good gang...
4        i watched it with my mom and we were like...<b...
                               ...                        
49964    this movie is terrible. it's about some no bra...
49965    well, what was fun... except for the fun part....
49966    by the time this film was released i had seen ...
49967    well, if you like pop/punk, punk, ska, and a t...
49968    where this movie is faithful to burroughs' vis...
Name: review, Length: 49969, dtype: object

In [5]:
#remove html tags
import re
def remove_html_tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub('', text)

In [6]:
df['review'].apply(remove_html_tags)

0        This movie is just crap. Even though the direc...
1        Another detailed work on the subject by Dr Dwi...
2        THE CAT O'NINE TAILS (Il Gatto a Nove Code) As...
3        Like with any movie genre, there are good gang...
4        I watched it with my mom and we were like...Wh...
                               ...                        
49964    This movie is terrible. It's about some no bra...
49965    Well, what was fun... except for the fun part....
49966    By the time this film was released I had seen ...
49967    Well, if you like pop/punk, punk, ska, and a t...
49968    Where this movie is faithful to Burroughs' vis...
Name: review, Length: 49969, dtype: object

In [7]:
#remove urls
def remove_url(text):
    pattern = re.compile((r'https?://\S+|www\.\S+'))
    return pattern.sub('', text)

In [8]:
text='Checkout this webpage to learn more about deep learning https://www.deeplearning.ai/ai-notes/initialization/index.html'

remove_url(text)

'Checkout this webpage to learn more about deep learning '

In [9]:
#remove punctuation
import string, time
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [10]:
exclude = string.punctuation

In [11]:
def remove_punc(text):
    for char in exclude:
        text = text.replace(char,'')
    return text
text = "okay,so . this , is/ just . a text ; with punc.?"
remove_punc(text)
    

'okayso  this  is just  a text  with punc'

In [12]:
start = time.time()
print(remove_punc(text))
time1= time.time()-start
print(time1)

okayso  this  is just  a text  with punc
0.00014066696166992188


In [13]:
#use this when you have large dataset
def remove_punc2(text):
    return text.translate(str.maketrans('', exclude))

remove_punc(text)

'okayso  this  is just  a text  with punc'

In [14]:
start = time.time()
print(remove_punc(text))
time1= time.time()-start
print(time1)

okayso  this  is just  a text  with punc
0.00035381317138671875


In [15]:
#Chat word treatment(GenZ lingo treatment)
chat_word = {
    'AFAIK': 'As Far As I Know',
    'AFK': 'Away From Keyboard',
    'ASAP': 'As Soon As Possible',
    'ATK': 'At The Keyboard',
    'ATM': 'At The Moment',
    'A3': 'Anytime, Anywhere, Anyplace',
    'BAK': 'Back At Keyboard',
    'BBL': 'Be Back Later',
    'BBS': 'Be Back Soon',
    'BFN': 'Bye For Now',
    'B4N': 'Bye For Now',
    'BRB': 'Be Right Back',
    'BRT': 'Be Right There',
    'BTW': 'By The Way',
    'B4': 'Before',
    'CU': 'See You',
    'CUL8R': 'See You Later',
    'CYA': 'See You',
    'FAQ': 'Frequently Asked Questions',
    'FC': 'Fingers Crossed',
    'FWIW': "For What It's Worth",
    'FYI': 'For Your Information',
    'GAL': 'Get A Life',
    'GG': 'Good Game',
    'GN': 'Good Night',
    'GMTA': 'Great Minds Think Alike',
    'GR8': 'Great!',
    'G9': 'Genius',
    'IC': 'I See',
    'ICQ': 'I Seek you (also a chat program)',
    'ILU': 'ILU: I Love You',
    'IMHO': 'In My Honest/Humble Opinion',
    'IMO': 'In My Opinion',
    'IOW': 'In Other Words',
    'IRL': 'In Real Life',
    'KISS': 'Keep It Simple, Stupid',
    'LDR': 'Long Distance Relationship',
    'LMAO': 'Laugh My A.. Off',
    'LOL': 'Laughing Out Loud',
    'LTNS': 'Long Time No See',
    'L8R': 'Later',
    'MTE': 'My Thoughts Exactly',
    'M8': 'Mate',
    'NRN': 'No Reply Necessary',
    'OIC': 'Oh I See',
    'PITA': 'Pain In The A..',
    'PRT': 'Party',
    'PRW': 'Parents Are Watching',
    'QPSA?': 'Que Pasa?',
    'ROFL': 'Rolling On The Floor Laughing',
    'ROFLOL': 'Rolling On The Floor Laughing Out Loud',
    'ROTFLMAO': 'Rolling On The Floor Laughing My A.. Off',
    'SK8': 'Skate',
    'STATS': 'Your sex and age',
    'ASL': 'Age, Sex, Location',
    'THX': 'Thank You',
    'TTFN': 'Ta-Ta For Now!',
    'TTYL': 'Talk To You Later',
    'U': 'You',
    'U2': 'You Too',
    'U4E': 'Yours For Ever',
    'WB': 'Welcome Back',
    'WTF': 'What The F...',
    'WTG': 'Way To Go!',
    'WUF': 'Where Are You From?',
    'W8': 'Wait...',
    '7K': 'Sick:-D Laugher',
    'TFW': 'That feeling when',
    'MFW': 'My face when',
    'MRW': 'My reaction when',
    'IFYP': 'I feel your pain',
    'TNTL': 'Trying not to laugh',
    'JK': 'Just kidding',
    'IDC': "I don't care",
    'ILY': 'I love you',
    'IMU': 'I miss you',
    'ADIH': 'Another day in hell',
    'ZZZ': 'Sleeping, bored, tired',
    'WYWH': 'Wish you were here',
    'TIME': 'Tears in my eyes',
    'BAE': 'Before anyone else',
    'FIMH': 'Forever in my heart',
    'BSAAW': 'Big smile and a wink',
    'BWL': 'Bursting with laughter',
    'BFF': 'Best friends forever',
    'CSL': "Can't stop laughing"
}

In [16]:
def short_con(text):
    new_text = []
    for w in text.split():
        if w.upper()in chat_word:
            new_text.append(chat_word[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)
            

In [17]:
short_con("ASAP Let me know")

'As Soon As Possible Let me know'

In [18]:
#spelling correction
from textblob import TextBlob
incorrect_text = "checki hoew welli your moedel perform"
txtblb = TextBlob(incorrect_text)
txtblb.correct().string

'check how well your model perform'

In [19]:
#removing stop words
from nltk.corpus import stopwords

In [20]:
stopwords.words('english')

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 'her',
 'here',
 'hers',
 'herself',
 "he's",
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 'if',
 "i'll",
 "i'm",
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 "i've",
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [21]:
def remove_stopwords(text):
    new_text = []
    for word in text.split():
        if word in stopwords.words('english'):
            new_text.append('')
        else:
            new_text.append(word)
    x = new_text[:]
    new_text.clear()
    return " ".join(x)

text = "It wasn't my fault"
remove_stopwords(text)

'It   fault'

In [22]:
#remove_emoji
import re
def remove_emoji(text):
    emoji_pattern=re.compile("["
                             u"\U0001F600-\U0001F64F" #emoticons
                             u"\U0001F300-\U0001F5FF" #symbols, pictograph
                              u"\U0001F680-\U0001F6FF" #transport and map symbol
                              u"\U0001F1E0-\U0001F1FF" # flags(IOS)
                              u"\U00002702-\U000027B0"
                              u"\U00002FC2-\U0001F251"
                             "]+",flags=re.UNICODE)
    return emoji_pattern.sub(r'',text)

text="that is not so funny please stop 😭"
remove_emoji(text)

'that is not so funny please stop '

In [23]:
#replace
import emoji
print(emoji.demojize(text))

that is not so funny please stop :loudly_crying_face:


**Tokenization**


In [24]:
#using split
text = 'Let\'s make a plan to go to rishikesh'
text.split()

["Let's", 'make', 'a', 'plan', 'to', 'go', 'to', 'rishikesh']

In [25]:
#peoblem where it fails, as it cannout seperate `!!!!!`
text1="I am, going to delhi!!!"
text1.split(',')

['I am', ' going to delhi!!!']

In [26]:
# Regular expression
import re
tokens=re.findall("[\w']+",text1)
tokens

['I', 'am', 'going', 'to', 'delhi']

In [27]:
#NLTK library
from nltk import word_tokenize, sent_tokenize
word_tokenize(text1)

['I', 'am', ',', 'going', 'to', 'delhi', '!', '!', '!']

In [28]:
text2='I have a Ph.D in M.L'
text3="We're here to help! mail us at xuz@gmail.com"

In [29]:
word_tokenize(text2)

['I', 'have', 'a', 'Ph.D', 'in', 'M.L']

In [30]:
# fails at this sentence as it seperated email id on the basis of @
word_tokenize(text3)

['We',
 "'re",
 'here',
 'to',
 'help',
 '!',
 'mail',
 'us',
 'at',
 'xuz',
 '@',
 'gmail.com']

In [31]:
#spacy
import spacy
nlp = spacy.load('en_core_web_sm')
doc1 = nlp(text1)
doc2 = nlp(text2)
doc3 = nlp(text3)

for i in doc3:
    print(i)

We
're
here
to
help
!
mail
us
at
xuz@gmail.com


In [32]:
#stemming
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
def stem_words(text):
    return " ".join([ps.stem(word)for word in text.split()])

sample = 'walk walking walked walks'
stem_words(sample)

'walk walk walk walk'

In [33]:
sample2='Stemming is a text preprocessing technique in NLP that reduces words to their root or base form by removing suffixes. Unlike lemmatization, which relies on linguistic rules, stemming uses heuristic methods to strip affixes. For example, "running," "runner," and "ran" might all be reduced to "run." Common algorithms include the Porter Stemmer, which applies a series of rules to iteratively strip suffixes, and the Snowball Stemmer, which is an improved version of the Porter Stemmer. While stemming is faster and simpler than lemmatization, it can be less accurate, sometimes producing stems that are not actual words.'
stem_words(sample2)

'stem is a text preprocess techniqu in nlp that reduc word to their root or base form by remov suffixes. unlik lemmatization, which reli on linguist rules, stem use heurist method to strip affixes. for example, "running," "runner," and "ran" might all be reduc to "run." common algorithm includ the porter stemmer, which appli a seri of rule to iter strip suffixes, and the snowbal stemmer, which is an improv version of the porter stemmer. while stem is faster and simpler than lemmatization, it can be less accurate, sometim produc stem that are not actual words.'

In [34]:
#lemmatization
nlp = spacy.load('en_core_web_sm')
sentence = "The children were playing in the park, running and laughing as they enjoyed their freedom, unaware of the time passing quickly by."
doc = nlp(sentence)

print("Original Word - Lemmatized Word")
for token in doc:
    print(f"{token.text} - {token.lemma_}")

Original Word - Lemmatized Word
The - the
children - child
were - be
playing - play
in - in
the - the
park - park
, - ,
running - run
and - and
laughing - laugh
as - as
they - they
enjoyed - enjoy
their - their
freedom - freedom
, - ,
unaware - unaware
of - of
the - the
time - time
passing - pass
quickly - quickly
by - by
. - .


# 2. Text Representation

# One hot encoding

One-hot encoding is a technique used to convert categorical data into a numerical format that can be used by machine learning algorithms. This method transforms each category value into a new binary column and assigns a 1 or 0 (True/False) to indicate the presence of that category in the data.

Advantages¶
Simplicity and Ease of Implementation:

One-hot encoding is straightforward to implement and understand, making it a go-to method for converting categorical data into numerical format.
No Ordinal Assumptions:

Unlike label encoding, one-hot encoding does not impose any ordinal relationship between categories, preserving the true nature of categorical data.
Compatibility with Machine Learning Algorithms:

Many machine learning algorithms, especially those that rely on distance calculations (e.g., k-nearest neighbors, linear regression), perform better with one-hot encoded data.
Avoidance of Bias:

Since each category is represented independently, there is no inherent bias introduced by the encoding process.

Disadvantages

High Dimensionality:

One-hot encoding can lead to a significant increase in the number of features, especially when dealing with categorical variables with many unique values. This high dimensionality can make the model computationally expensive and harder to train.

Sparse Data:

The resulting one-hot encoded data is sparse, containing many zeros. Sparse matrices can lead to inefficiencies in storage and processing.

Curse of Dimensionality:

The increase in dimensionality can lead to the curse of dimensionality, where the performance of machine learning algorithms degrades with the addition of too many features.

Scalability Issues:

For large datasets with high cardinality categorical variables, one-hot encoding may not be scalable and can result in memory and computational challenges.

Irrelevance in Some Algorithms:

Some machine learning algorithms, like decision trees, do not benefit from one-hot encoding as they can handle categorical data directly.


In [35]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

corpus = [
    'People Follow Mr.Beast',
    'Mr.Beast Follow Mr.Beast',
    'People Write comment',
    'Mr.Beast Write comment'
]

In [36]:
# Initialize CountVectorizer with binary=True for One-Hot Encoding
vectorizer = CountVectorizer(binary=True)

# Fit and transform the text data
X_onehot = vectorizer.fit_transform(corpus)

# Convert to a DataFrame for better readability
df_onehot = pd.DataFrame(X_onehot.toarray(), columns=vectorizer.get_feature_names_out())

# Show the result
print(df_onehot)

   beast  comment  follow  mr  people  write
0      1        0       1   1       1      0
1      1        0       1   1       0      0
2      0        1       0   0       1      1
3      1        1       0   1       0      1


# **Bag of words¶**


The core intuition behind the Bag of Words (BoW) model is that if our algorithm detects similar words occurring with similar frequencies, it will classify the documents as being in the same class. Here, the order of words and context do not matter; what matters is the presence and frequency of words.

In BoW, each word is represented as a coordinate in an n-dimensional space. If we set the binary parameter to True, any word that appears more than once is still treated as if it appears only once. Consequently, the coordinates will be binary, filled only with 0s and 1s.



In [37]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

corpus = [
    'People Follow Mr.Beast',
    'Mr.Beast Follow Mr.Beast',
    'People Write comment',
    'Mr.Beast Write comment'
]

In [38]:
vectorizer = CountVectorizer()
bow = vectorizer.fit_transform(corpus)
df = pd.DataFrame(bow.toarray(),columns = vectorizer.get_feature_names_out())
print(df)

   beast  comment  follow  mr  people  write
0      1        0       1   1       1      0
1      2        0       1   2       0      0
2      0        1       0   0       1      1
3      1        1       0   1       0      1


# N-Grams

In [39]:
#n-grams
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'People Follow Mr.Beast',
    'Mr.Beast Follow Mr.Beast',
    'People Write comment',
    'Mr.Beast Write comment'
]

In [40]:
cv = CountVectorizer(ngram_range=(3,3))
bow = cv.fit_transform(corpus)

In [41]:
print(cv.vocabulary_)

{'people follow mr': 5, 'follow mr beast': 2, 'mr beast follow': 3, 'beast follow mr': 0, 'people write comment': 6, 'mr beast write': 4, 'beast write comment': 1}


In [42]:
df = pd.DataFrame(bow.toarray(), columns = cv.get_feature_names_out())
print(df)

   beast follow mr  beast write comment  follow mr beast  mr beast follow  \
0                0                    0                1                0   
1                1                    0                1                1   
2                0                    0                0                0   
3                0                    1                0                0   

   mr beast write  people follow mr  people write comment  
0               0                 1                     0  
1               0                 0                     0  
2               0                 0                     1  
3               1                 0                     0  


# **TF-IDF**


In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

corpus = [
    'People Follow Mr.Beast',
    'Mr.Beast Follow Mr.Beast',
    'People Write comment',
    'Mr.Beast Write comment'
]

In [44]:
# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# vectorizer.fit_transform(corpus).toarray()
# Fit and transform
X_tfidf = vectorizer.fit_transform(corpus)

# Convert to DataFrame
df_tfidf = pd.DataFrame(X_tfidf.toarray(), columns=vectorizer.get_feature_names_out())

# Round for cleaner display
print(df_tfidf.round(2))

   beast  comment  follow    mr  people  write
0   0.44     0.00    0.55  0.44    0.55   0.00
1   0.65     0.00    0.40  0.65    0.00   0.00
2   0.00     0.58    0.00  0.00    0.58   0.58
3   0.44     0.55    0.00  0.44    0.00   0.55


In [45]:
# print(vectorizer.idf_)
# print(vectorizer.get_feature_names_out())

# Word embedding

**Word2vec**

In [46]:
!pip install gensim


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 44.6 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.2
    Uninstalling scipy-1.15.2:
      Successfully uninstalled scipy-1.15.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you have gymnasium 0.29.0 which is incompatible.
imbalanced-learn 0.13.0 requires scikit-learn<2,>=1.3.2, but you have scikit-learn 1.2.2 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.2 which is incompatible.
mlxte

In [47]:
import gensim
from gensim.models import Word2Vec, KeyedVectors

In [48]:
import gensim.downloader
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [49]:
#method1
# w2v = gensim.downloader.load('glove-twitter-25')

In [50]:
#method2
import gensim.downloader as api
w2v = api.load('glove-twitter-25')

[==================================================] 100.0% 104.8/104.8MB downloaded


In [51]:
print(w2v['happy'])

[-1.2304   0.48312  0.14102 -0.0295  -0.65253 -0.18554  2.1033   1.7516
 -1.3001  -0.32113 -0.84774  0.41995 -3.8823   0.19638 -0.72865 -0.85273
  0.23174 -1.0763  -0.83023  0.10815 -0.51015  0.27691 -1.1895   0.98094
 -0.13955]


In [52]:
w2v.most_similar('king')

[('prince', 0.9337410926818848),
 ('queen', 0.920242190361023),
 ('aka', 0.9176921844482422),
 ('lady', 0.9163240194320679),
 ('jack', 0.9147354960441589),
 ("'s", 0.9066898226737976),
 ('stone', 0.8982374668121338),
 ('mr.', 0.8919409513473511),
 ('the', 0.8893438577651978),
 ('star', 0.8892088532447815)]

In [53]:
w2v['cricket']

array([ 0.48169 , -0.066325, -0.44454 , -1.7274  , -1.0247  , -0.78493 ,
        0.50351 , -0.65595 ,  0.56451 , -0.03321 , -0.14383 ,  0.71937 ,
       -2.8215  ,  0.86131 ,  0.79783 , -0.49762 ,  0.044193, -0.24435 ,
        0.051926,  1.0575  , -1.1741  ,  0.84042 ,  0.35934 , -1.0306  ,
       -0.69021 ], dtype=float32)

In [54]:
w2v.most_similar('batman')

[('thor', 0.9278313517570496),
 ('potter', 0.9092318415641785),
 ('superman', 0.9090892672538757),
 ('spiderman', 0.9049674272537231),
 ('gangnam', 0.8759423494338989),
 ('hobbit', 0.8744966387748718),
 ('transformers', 0.8698741793632507),
 ('bob', 0.8696600794792175),
 ('barney', 0.8690287470817566),
 ('disney', 0.8650587797164917)]

In [55]:
w2v.most_similar('cricket')

[('ncaa', 0.9297444224357605),
 ('ipl', 0.9059461355209351),
 ('nhl', 0.893334150314331),
 ('championship', 0.8847678899765015),
 ('england', 0.8797493577003479),
 ('wins', 0.8647025227546692),
 ('indy', 0.8642993569374084),
 ('auburn', 0.8611217141151428),
 ('tna', 0.8553740382194519),
 ('basketball', 0.8542302250862122)]

In [56]:
w2v.most_similar('facebook')

[('twitter', 0.9480051398277283),
 ('google', 0.9231430888175964),
 ('instagram', 0.9184154272079468),
 ('internet', 0.914387583732605),
 ('youtube', 0.9113808274269104),
 ('tumblr', 0.9077149033546448),
 ('link', 0.8995786905288696),
 ('fb', 0.8734269142150879),
 ('post', 0.8671452403068542),
 ('site', 0.8642723560333252)]

In [57]:
w2v.similarity('man', 'woman')

0.76541775

In [58]:
w2v.doesnt_match(["coffee", "spiderman", "batman"])

'coffee'

In [59]:
vec = w2v['king'] - w2v['man'] + w2v['woman']
w2v.most_similar([vec])

[('meets', 0.8459880352020264),
 ('crow', 0.7845019102096558),
 ('voices', 0.7781485915184021),
 ('woman', 0.7778489589691162),
 ('widow', 0.772538959980011),
 ('mercy', 0.7720795273780823),
 ('soldier', 0.7719036340713501),
 ('shepherd', 0.7657868266105652),
 ('hunter', 0.7621815800666809),
 ('treasure', 0.7620967030525208)]

**CBOW**

In [60]:
import gensim
import os


In [61]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

story = []
for filename in os.listdir('/kaggle/input/harry-potter-books'):
    f = open(os.path.join('/kaggle/input/harry-potter-books', filename))
    corpus = f.read()
    raw_sent = sent_tokenize(corpus)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [62]:
len(story)

66129

In [63]:
model = gensim.models.Word2Vec(
    window = 10,
    min_count = 2
)
model.build_vocab(story)
model.train(story, total_examples = model.corpus_count, epochs= model.epochs)

(4029505, 5322610)

In [64]:
model.wv.most_similar('voldemort')

[('sirius', 0.7580131888389587),
 ('dumbledore', 0.7107201814651489),
 ('curse', 0.661498486995697),
 ('riddle', 0.6396136283874512),
 ('prickle', 0.6363064646720886),
 ('kreacher', 0.6337164044380188),
 ('snape', 0.6333354115486145),
 ('he', 0.6246486902236938),
 ('hallow', 0.622176468372345),
 ('dark', 0.6214813590049744)]

In [65]:
model.wv.doesnt_match(['voldemort','harry','dumbledore','hermione','draco'])

'draco'

In [66]:
model.wv['wizard']

array([-0.803939  ,  1.8172108 , -0.51290447,  0.07299227, -1.0815089 ,
       -0.56315684,  0.5595186 ,  0.4042707 , -0.49713528, -0.72057015,
        0.10844541,  0.9701403 , -0.05471553, -0.3120165 , -0.43067488,
       -0.30056286, -0.5946476 ,  0.15371805, -0.5262246 ,  0.16316524,
       -0.3761015 ,  0.5792903 ,  1.2212151 , -0.46809027, -0.62275857,
        0.35409248, -0.6940376 ,  0.45439512, -0.08415239, -0.5544289 ,
        0.05855036, -1.0015395 ,  0.05189426, -0.0254657 ,  0.56134933,
       -0.6503336 , -0.11529888, -0.7719715 ,  0.38953757, -0.71073514,
       -1.4632365 ,  0.3965471 ,  0.2933467 , -0.9654306 ,  0.5281116 ,
       -0.9300672 , -0.84696686, -0.2339494 ,  0.07206122,  0.4677139 ,
        0.18654487, -0.8320747 , -0.70837206, -0.7580091 , -0.07178392,
        0.4619997 ,  0.9937288 , -0.07422944, -0.4018687 , -0.05385853,
       -0.29969925, -0.74405813, -0.42216572,  0.18229784, -1.0561249 ,
       -0.09134937, -0.5045118 ,  0.37648752, -0.9583148 ,  0.98

In [67]:
model.wv.similarity('hermione','harry')

0.3676886

In [68]:
model.wv.similarity('wizard','magic')

0.63416326

In [69]:
model.wv.get_normed_vectors().shape

(14675, 100)

In [70]:
y = model.wv.index_to_key
y

['the',
 'and',
 'to',
 'he',
 'of',
 'harry',
 'was',
 'you',
 'it',
 'said',
 'his',
 'in',
 'that',
 'had',
 'at',
 'as',
 'on',
 'him',
 'they',
 'with',
 'ron',
 'but',
 'for',
 'her',
 'not',
 'hermione',
 'she',
 'what',
 'be',
 'out',
 'up',
 'all',
 'were',
 'we',
 'have',
 'them',
 'from',
 'there',
 'into',
 'dumbledore',
 'who',
 'been',
 'this',
 'back',
 'so',
 'is',
 'me',
 'could',
 'no',
 'if',
 'now',
 'an',
 'one',
 'then',
 'their',
 'about',
 'know',
 'over',
 'like',
 'down',
 'looked',
 'when',
 'do',
 'just',
 'would',
 'did',
 'by',
 'around',
 've',
 'your',
 'got',
 'very',
 'are',
 'can',
 'hagrid',
 'professor',
 'my',
 'don',
 'more',
 'well',
 're',
 'see',
 'again',
 'off',
 'snape',
 'though',
 'how',
 'looking',
 'think',
 'll',
 'time',
 'still',
 'get',
 'wand',
 'right',
 'eyes',
 'or',
 'weasley',
 'face',
 'through',
 'which',
 'going',
 'before',
 'voice',
 'here',
 'look',
 'go',
 'didn',
 'where',
 'than',
 'will',
 'room',
 'only',
 'potter',


In [71]:
from sklearn.decomposition import PCA

In [72]:
pca = PCA(n_components = 3)

In [73]:
X = pca.fit_transform(model.wv.get_normed_vectors())

In [74]:
X[:5]

array([[-0.09303354,  0.6938841 , -0.1992675 ],
       [-0.27488008,  0.41478443,  0.13788348],
       [ 0.36112523,  0.5986741 ,  0.22247545],
       [ 0.21638526,  0.59233314,  0.31458002],
       [ 0.03547409,  0.46441072, -0.12554611]], dtype=float32)

In [75]:
X.shape

(14675, 3)

In [76]:
import pandas as pd
import plotly.express as px

# Select a range of rows from X and y
# X_subset = X[100:300]  # Assuming X has at least 300 rows and 3 columns
# y_subset = y[100:300]  # Ensure y has the same number of elements as the selected range in X

# # Create a DataFrame from the subset of X
# df = pd.DataFrame(X_subset, columns=['x', 'y', 'z'])

# # Create the 3D scatter plot
# fig = px.scatter_3d(df, x='x', y='y', z='z', color=y_subset)
# fig.show()

fig = px.scatter_3d(X[200:300], x =0, y=1, z=2, color = y[200:300])
fig.show()

# Text classification

POS

In [77]:
!pip install spacy

In [78]:
import spacy

In [79]:
nlp = spacy.load('en_core_web_sm')

In [80]:
doc = nlp('inder is very hardworking girl')

In [81]:
doc.text

'inder is very hardworking girl'

In [82]:
doc[3]

hardworking

In [83]:

doc[1].pos_

'AUX'

In [84]:
doc[0].pos_

'NOUN'

In [85]:
doc[3].tag_

'VBG'

In [86]:
spacy.explain('VBG')

'verb, gerund or present participle'

In [87]:
for word in doc:
    print(word.text,"----->", word.pos_, word.tag_,spacy.explain(word.tag_))

inder -----> NOUN NN noun, singular or mass
is -----> AUX VBZ verb, 3rd person singular present
very -----> ADV RB adverb
hardworking -----> VERB VBG verb, gerund or present participle
girl -----> NOUN NN noun, singular or mass


In [88]:
doc2 = nlp("I want to be a Data scientist")

In [89]:
for word in doc2:
    print(word.text,"------>", word.pos_, word.tag_, spacy.explain(word.tag_))

I ------> PRON PRP pronoun, personal
want ------> VERB VBP verb, non-3rd person singular present
to ------> PART TO infinitival "to"
be ------> AUX VB verb, base form
a ------> DET DT determiner
Data ------> PROPN NNP noun, proper singular
scientist ------> NOUN NN noun, singular or mass


In [90]:
doc3 = nlp("i have read a book on politics")


In [91]:
for word in doc3:
    print(word.text,'------>', word.pos_, word.tag_, spacy.explain(word.tag_))

i ------> PRON PRP pronoun, personal
have ------> AUX VBP verb, non-3rd person singular present
read ------> VERB VBN verb, past participle
a ------> DET DT determiner
book ------> NOUN NN noun, singular or mass
on ------> ADP IN conjunction, subordinating or preposition
politics ------> NOUN NNS noun, plural


In [92]:
doc4 = nlp('The quick brown fox jumped over the lazy dog')


In [93]:
from spacy import displacy
#for visualization

In [94]:
displacy.render(doc4, style= 'dep', jupyter = True)

In [95]:
options={
    'distance': 80,
    'compact': True,
    'color': '#fff',
    'bg': 'green'
    
}

In [96]:
displacy.render(doc4, style = 'dep', jupyter = True, options = options)